<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/BERT_FINAL_FAE_ANON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langdetect deep_translator faker tqdm transformers torch pandas

import sqlite3
import pandas as pd
import re
import time
from langdetect import detect, DetectorFactory, LangDetectException
from deep_translator import GoogleTranslator
from faker import Faker
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from transformers import pipeline

# Configuración inicial
DetectorFactory.seed = 0
fake_es = Faker("es_ES")

# Cargar modelo BERT para NER en español
ner_model = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")


def detectar_genero(nombre):
    """Detecta si un nombre es masculino o femenino según Faker"""
    if nombre in fake_es.first_name_male():
        return "male"
    elif nombre in fake_es.first_name_female():
        return "female"
    return "neutral"


def generar_nombre_con_palabras(original_name):
    """Genera un nombre falso con el mismo número de palabras y el mismo género que el original"""
    palabras = original_name.split()
    num_palabras = len(palabras)

    # Determinar el género del primer nombre
    genero = detectar_genero(palabras[0])

    while True:
        if num_palabras == 1:
            nombre = fake_es.first_name_male() if genero == "male" else fake_es.first_name_female()
        elif num_palabras == 2:
            nombre = f"{fake_es.first_name_male() if genero == 'male' else fake_es.first_name_female()} {fake_es.last_name()}"
        else:
            nombre = f"{fake_es.first_name_male() if genero == 'male' else fake_es.first_name_female()} {fake_es.last_name()} {fake_es.last_name()}"

        if len(nombre.split()) == num_palabras:
            return nombre


def detectar_entidades(texto):
    """Detecta entidades PER usando BERT"""
    resultados = ner_model(texto)
    entidades = []

    for ent in resultados:
        if ent['entity_group'] == 'PER':
            start = ent['start']
            end = ent['end']
            original = texto[start:end]

            # Ajuste fino para capturar correctamente los espacios
            while start > 0 and texto[start-1] != ' ':
                start -= 1
            while end < len(texto) and texto[end] != ' ':
                end += 1

            entidades.append((start, end, original.strip()))

    return entidades


def traducir_y_anonimizar(texto):
    """Traduce y anonimiza manteniendo el número de palabras y el género"""
    if not texto.strip():
        return texto

    # Traducción
    try:
        if len(texto) > 3 and detect(texto) != "es":
            texto = GoogleTranslator(source="auto", target="es").translate(texto)
    except LangDetectException:
        pass

    # Anonimizar nombres en "Me llamo..."
    texto = re.sub(
        r"(Me llamo\s+)([A-ZÁÉÍÓÚÑa-záéíóúñ]+(?:\s+[A-ZÁÉÍÓÚÑa-záéíóúñ]+)*)",
        lambda match: match.group(1) + generar_nombre_con_palabras(match.group(2)),
        texto,
        flags=re.IGNORECASE
    )

    # Anonimizar números
    texto = re.sub(r"\b\d{9}\b", lambda _: fake_es.phone_number(), texto)
    texto = re.sub(r"\b\d{8}[A-Za-z]\b", lambda _: fake_es.ssn(), texto)

    # Detección de entidades con BERT
    entidades = detectar_entidades(texto)
    replacements = []

    for start, end, original in entidades:
        fake_name = generar_nombre_con_palabras(original)
        replacements.append((start, end, fake_name))

    # Aplicar reemplazos en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    return texto

def procesar_fila(row):
    """Procesa una fila aplicando traducción y anonimización."""
    row["body"] = traducir_y_anonimizar(row["body"])
    return row

# Conectar a la base de datos
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    # Crear la tabla si no existe
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS mi_tabla (
            id INTEGER PRIMARY KEY,
            body TEXT,
            secret TEXT,
            direction TEXT,
            createdAt TEXT,
            OpenchannelAccountId INTEGER,
            OpenchannelInteractionId INTEGER,
            UserId INTEGER,
            ContactId INTEGER,
            AttachmentId INTEGER,
            sentBy TEXT
        );
    """)
    conn.commit()

    # Cargar datos desde Excel (solo si es necesario)
    df = pd.read_excel("MOSTRA_1.xlsx")
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    # Extraer solo columnas relevantes
    df_body = df[["id", "body", "direction","createdAt","UserId", "ContactId"]].copy()

    # Usar tqdm para mostrar progreso
    start_time = time.time()
    with Pool(cpu_count()) as pool:
        result = list(tqdm(pool.imap(procesar_fila, df_body.to_dict(orient="records")), total=len(df_body)))

    # Convertir la lista de diccionarios de vuelta a DataFrame
    df_body = pd.DataFrame(result)

    # Actualizar base de datos en un solo paso eficiente
    df_body.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    elapsed_time = time.time() - start_time
    print(f"Procesamiento completado en {elapsed_time:.2f} segundos.")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initial

Procesamiento completado en 589.33 segundos.


In [ ]:
df_body = df_body.dropna(subset=["UserId"])

In [ ]:
# prompt: 2 dataframes based on the variable direction in and out

import pandas as pd
# Create two dataframes based on the 'direction' column
df_in = df_body[df_body['direction'] == 'in'].copy()
df_out = df_body[df_body['direction'] == 'out'].copy()

# Now you have two separate dataframes: df_in and df_out
print("DataFrame 'in':")
print(df_in.head())  # Display the first few rows of df_in
print("\nDataFrame 'out':")
print(df_out.head()) # Display the first few rows of df_out


DataFrame 'in':
      id                 body direction           createdAt  UserId  ContactId
0  77623                 Hola        in 2023-01-01 00:10:19   109.0        466
2  77625    Me llamo Aránzazu        in 2023-01-01 00:10:37   109.0        466
5  77628               Acepto        in 2023-01-01 00:12:13   109.0        466
7  77630  Feliz año nuevo 🎆🎈🎊        in 2023-01-01 00:14:09   109.0        466
8  77631                🥰🥰🥰🥰🥰        in 2023-01-01 00:14:17   109.0        466

DataFrame 'out':
       id                                               body direction  \
3   77626                                              Hola!       out   
4   77627  Antes de empezar a chatear, es necesario que l...       out   
6   77629  Gracias Graciana Soy maria jose, en que te pue...       out   
10  77633  Feliz año nuevo! Que 2023 te traiga cosas buen...       out   
13  77636                          des de donde me escribes?       out   

             createdAt  UserId  ContactId  
3  

In [ ]:
# prompt: 20 lineas del dataframe df_body

# Display the first 20 lines of the df_body DataFrame
print(df_body.head(20))


       id                                               body direction  \
0   77623                                               Hola        in   
2   77625                                  Me llamo Aránzazu        in   
3   77626                                              Hola!       out   
4   77627  Antes de empezar a chatear, es necesario que l...       out   
5   77628                                             Acepto        in   
6   77629  Gracias Graciana Soy maria jose, en que te pue...       out   
7   77630                                Feliz año nuevo 🎆🎈🎊        in   
8   77631                                              🥰🥰🥰🥰🥰        in   
9   77632  Siento sola por eso dame la mensaje para nuevo...        in   
10  77633  Feliz año nuevo! Que 2023 te traiga cosas buen...       out   
11  77634                                            Gracias        in   
12  77635                                              🥰🥰🥰🥰🥰        in   
13  77636                          des

In [1]:
!pip install pyspellchecker # Install pyspellchecker instead of spellchecker
!pip install langdetect deep_translator faker tqdm transformers torch pandas emoji
import sqlite3
import pandas as pd
import re
import time
from langdetect import detect, DetectorFactory, LangDetectException
from deep_translator import GoogleTranslator
from faker import Faker
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from transformers import pipeline
import emoji
from spellchecker import SpellChecker # Import SpellChecker from pyspellchecker instead of spellchecker

# Configuración inicial
DetectorFactory.seed = 0
fake_es = Faker("es_ES")
spell = SpellChecker(language="es")

# Cargar modelo BERT para NER en español
ner_model = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")

# Diccionario para mantener consistencia de nombres inventados
nombre_map = {}

def detectar_genero(nombre):
    """Detecta si un nombre es masculino o femenino según Faker"""
    if nombre in fake_es.first_name_male():
        return "male"
    elif nombre in fake_es.first_name_female():
        return "female"
    return "neutral"

def generar_nombre_con_palabras(original_name):
    """Genera un nombre falso con el mismo número de palabras y el mismo género"""
    palabras = original_name.split()
    num_palabras = len(palabras)

    # Si ya existe un nombre ficticio asignado, reutilizarlo
    if original_name in nombre_map:
        return nombre_map[original_name]

    # Determinar género
    genero = detectar_genero(palabras[0])

    while True:
        if num_palabras == 1:
            nombre = fake_es.first_name_male() if genero == "male" else fake_es.first_name_female()
        elif num_palabras == 2:
            nombre = f"{fake_es.first_name_male() if genero == 'male' else fake_es.first_name_female()} {fake_es.last_name()}"
        else:
            nombre = f"{fake_es.first_name_male() if genero == 'male' else fake_es.first_name_female()} {fake_es.last_name()} {fake_es.last_name()}"

        if len(nombre.split()) == num_palabras:
            nombre_map[original_name] = nombre  # Guardar para mantener consistencia
            return nombre

def detectar_entidades(texto):
    """Detecta entidades PER usando BERT"""
    resultados = ner_model(texto)
    entidades = []

    for ent in resultados:
        if ent['entity_group'] == 'PER':
            start = ent['start']
            end = ent['end']
            original = texto[start:end]

            # Ajuste fino para capturar correctamente los espacios
            while start > 0 and texto[start-1] != ' ':
                start -= 1
            while end < len(texto) and texto[end] != ' ':
                end += 1

            entidades.append((start, end, original.strip()))

    return entidades

def normalizar_texto(texto):
    """Convierte el texto a minúsculas, corrige errores ortográficos y elimina emojis"""
    texto = texto.lower()  # Convertir a minúsculas
    texto = emoji.replace_emoji(texto, replace="")  # Eliminar emojis
    texto = re.sub(r"[^A-Za-zÁÉÍÓÚáéíóúÑñ0-9.,!?;:\s]", "", texto)  # Eliminar caracteres extraños
    texto = " ".join([spell.correction(word) if spell.correction(word) else word for word in texto.split()])  # Corregir ortografía
    return texto.strip()

def traducir_y_anonimizar(texto):
    """Traduce y anonimiza manteniendo consistencia, número de palabras y género"""
    if not texto.strip():
        return texto

    # Preprocesar texto antes de anonimizar
    texto = normalizar_texto(texto)

    # Traducción si es necesario
    try:
        if len(texto) > 3 and detect(texto) != "es":
            texto = GoogleTranslator(source="auto", target="es").translate(texto)
    except LangDetectException:
        pass

    # Anonimizar nombres en "Me llamo..."
    texto = re.sub(
        r"(me llamo\s+)([a-záéíóúñ]+(?:\s+[a-záéíóúñ]+)*)",
        lambda match: match.group(1) + generar_nombre_con_palabras(match.group(2)),
        texto,
        flags=re.IGNORECASE
    )

    # Anonimizar números
    texto = re.sub(r"\b\d{9}\b", lambda _: fake_es.phone_number(), texto)
    texto = re.sub(r"\b\d{8}[A-Za-z]\b", lambda _: fake_es.ssn(), texto)

    # Detección de entidades con BERT
    entidades = detectar_entidades(texto)
    replacements = []

    for start, end, original in entidades:
        fake_name = generar_nombre_con_palabras(original)
        replacements.append((start, end, fake_name))

    # Aplicar reemplazos en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    return texto


def procesar_fila(row):
    """Procesa una fila aplicando traducción y anonimización."""
    row["body"] = traducir_y_anonimizar(row["body"])
    return row

# Conectar a la base de datos
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    # Crear la tabla si no existe
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS mi_tabla (
            id INTEGER PRIMARY KEY,
            body TEXT,
            secret TEXT,
            direction TEXT,
            createdAt TEXT,
            OpenchannelAccountId INTEGER,
            OpenchannelInteractionId INTEGER,
            UserId INTEGER,
            ContactId INTEGER,
            AttachmentId INTEGER,
            sentBy TEXT
        );
    """)
    conn.commit()

    # Cargar datos desde Excel (solo si es necesario)
    df = pd.read_excel("MOSTRA_1.xlsx")
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    # Extraer solo columnas relevantes
    df_body = df[["id", "body", "direction","createdAt","UserId", "ContactId"]].copy()

    # Usar tqdm para mostrar progreso
    start_time = time.time()
    with Pool(cpu_count()) as pool:
        result = list(tqdm(pool.imap(procesar_fila, df_body.to_dict(orient="records")), total=len(df_body)))

    # Convertir la lista de diccionarios de vuelta a DataFrame
    df_body = pd.DataFrame(result)

    # Actualizar base de datos en un solo paso eficiente
    df_body.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    elapsed_time = time.time() - start_time
    print(f"Procesamiento completado en {elapsed_time:.2f} segundos.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]


  0%|          | 0/6943 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

  0%|          | 1/6943 [00:01<3:25:39,  1.78s/it]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

 38%|███▊      | 2642/6943 [23:43<38:36,  1.86it/s]Process ForkPoolWorker-1:
Process ForkPoolWorker-2:



KeyboardInterrupt: 

In [4]:
import pandas as pd
import random
import re

# Cargar lista de nombres desde el archivo CSV
nombres_df = pd.read_csv("2024_pad_m_nom_sexe.csv")

# Filtrar nombres por género (convertir a minúsculas y eliminar espacios extra)
nombres_masculinos = set(nombres_df[nombres_df["SEXE"] == 2]["NOM"].str.strip().str.lower().tolist())
nombres_femeninos = set(nombres_df[nombres_df["SEXE"] == 1]["NOM"].str.strip().str.lower().tolist())

# Diccionario para mantener consistencia de nombres inventados
nombre_map = {}

def detectar_genero(nombre):
    """Detecta si un nombre es masculino o femenino basado en la lista cargada."""
    nombre = nombre.lower()
    if nombre in nombres_masculinos:
        return "male"
    elif nombre in nombres_femeninos:
        return "female"
    return "neutral"

def mantener_formato(original, nuevo):
    """Ajusta el formato del nuevo nombre para que coincida con el original."""
    if original.istitle():  # Primera letra mayúscula
        return nuevo.title()
    elif original.isupper():  # Todo en mayúsculas
        return nuevo.upper()
    else:  # Todo en minúsculas
        return nuevo.lower()

def generar_nombre(original_name):
    """Genera un nombre ficticio manteniendo el género y la cantidad de palabras."""
    palabras = original_name.split()
    num_palabras = len(palabras)

    if original_name in nombre_map:
        return nombre_map[original_name]

    genero = detectar_genero(palabras[0])
    nuevo_nombre = []

    for _ in range(num_palabras):
        if genero == "male":
            nuevo_nombre.append(random.choice(list(nombres_masculinos)))
        else:
            nuevo_nombre.append(random.choice(list(nombres_femeninos)))

    nombre_final = " ".join(nuevo_nombre)
    nombre_final = mantener_formato(original_name, nombre_final)

    nombre_map[original_name] = nombre_final
    return nombre_final

def anonimizar_texto(texto):
    """Reemplaza nombres en el texto por versiones anonimizadas manteniendo consistencia."""
    palabras = texto.split()
    texto_anonimizado = []

    for palabra in palabras:
        palabra_limpia = re.sub(r'[^A-Za-zÁÉÍÓÚáéíóúÑñ]', '', palabra).lower()
        if palabra_limpia in nombres_masculinos or palabra_limpia in nombres_femeninos:
            texto_anonimizado.append(generar_nombre(palabra))
        else:
            texto_anonimizado.append(palabra)

    return " ".join(texto_anonimizado)

# Cargar datos desde el archivo Excel
df = pd.read_excel("MOSTRA_1.xlsx")

# Aplicar anonimización a la columna 'body'
df["body"] = df["body"].astype(str).apply(anonimizar_texto)

# Guardar el archivo anonimizado
df.to_excel("MOSTRA_1_anonimizado.xlsx", index=False)




In [5]:
import pandas as pd
import random
import re

# Cargar lista de nombres desde el archivo CSV
nombres_df = pd.read_csv("2024_pad_m_nom_sexe.csv")

# Filtrar nombres por género (convertir a minúsculas y eliminar espacios extra)
nombres_masculinos = set(nombres_df[nombres_df["SEXE"] == 2]["NOM"].str.strip().str.lower().tolist())
nombres_femeninos = set(nombres_df[nombres_df["SEXE"] == 1]["NOM"].str.strip().str.lower().tolist())

# Diccionario para mantener consistencia de nombres inventados
nombre_map = {}
cambios = []  # Lista para registrar los cambios

def detectar_genero(nombre):
    """Detecta si un nombre es masculino o femenino basado en la lista cargada."""
    nombre = nombre.lower()
    if nombre in nombres_masculinos:
        return "male"
    elif nombre in nombres_femeninos:
        return "female"
    return "neutral"

def mantener_formato(original, nuevo):
    """Ajusta el formato del nuevo nombre para que coincida con el original."""
    if original.istitle():  # Primera letra mayúscula
        return nuevo.title()
    elif original.isupper():  # Todo en mayúsculas
        return nuevo.upper()
    else:  # Todo en minúsculas
        return nuevo.lower()

def generar_nombre(original_name):
    """Genera un nombre ficticio manteniendo el género y la cantidad de palabras."""
    palabras = original_name.split()
    num_palabras = len(palabras)

    if original_name in nombre_map:
        return nombre_map[original_name]

    genero = detectar_genero(palabras[0])
    nuevo_nombre = []

    for _ in range(num_palabras):
        if genero == "male":
            nuevo_nombre.append(random.choice(list(nombres_masculinos)))
        else:
            nuevo_nombre.append(random.choice(list(nombres_femeninos)))

    nombre_final = " ".join(nuevo_nombre)
    nombre_final = mantener_formato(original_name, nombre_final)

    nombre_map[original_name] = nombre_final
    return nombre_final

def anonimizar_texto(texto):
    """Reemplaza nombres en el texto por versiones anonimizadas manteniendo consistencia."""
    palabras = texto.split()
    texto_anonimizado = []

    for i, palabra in enumerate(palabras):
        palabra_limpia = re.sub(r'[^A-Za-zÁÉÍÓÚáéíóúÑñ]', '', palabra).lower()
        if palabra_limpia in nombres_masculinos or palabra_limpia in nombres_femeninos:
            nombre_anonimizado = generar_nombre(palabra)
            texto_anonimizado.append(nombre_anonimizado)
            palabra_anterior = palabras[i - 1] if i > 0 else ""
            palabra_posterior = palabras[i + 1] if i < len(palabras) - 1 else ""
            cambios.append((palabra, nombre_anonimizado, palabra_anterior, palabra_posterior))
        else:
            texto_anonimizado.append(palabra)

    return " ".join(texto_anonimizado)

# Cargar datos desde el archivo Excel
df = pd.read_excel("MOSTRA_1.xlsx")

# Aplicar anonimización a la columna 'body'
df["body"] = df["body"].astype(str).apply(anonimizar_texto)

# Guardar el archivo anonimizado
df.to_excel("MOSTRA_1_anonimizado.xlsx", index=False)

# Crear DataFrame con los cambios y guardarlo en un archivo separado
cambios_df = pd.DataFrame(cambios, columns=["Original", "Anonimizado", "Palabra_Anterior", "Palabra_Posterior"])
cambios_df.to_excel("cambios_anonimizacion.xlsx", index=False)


In [6]:
import pandas as pd
import random
import re

# Cargar lista de nombres desde el archivo CSV
nombres_df = pd.read_csv("2024_pad_m_nom_sexe.csv")

# Filtrar nombres por género (convertir a minúsculas y eliminar espacios extra)
nombres_masculinos = set(nombres_df[nombres_df["SEXE"] == 2]["NOM"].str.strip().str.lower().tolist())
nombres_femeninos = set(nombres_df[nombres_df["SEXE"] == 1]["NOM"].str.strip().str.lower().tolist())

# Lista de palabras comunes que no deben anonimizarse
palabras_excluidas = {"bo","d'una","dan","justo","salud","li","leo","te","ella", "el", "los", "las", "nosotros", "vosotros", "usted", "ustedes", "ellos", "ellas", "su", "sus", "un", "una", "unos", "unas", "nada"}

# Diccionario para mantener consistencia de nombres inventados
nombre_map = {}
cambios = []  # Lista para registrar los cambios

def detectar_genero(nombre):
    """Detecta si un nombre es masculino o femenino basado en la lista cargada."""
    nombre = nombre.lower()
    if nombre in nombres_masculinos:
        return "male"
    elif nombre in nombres_femeninos:
        return "female"
    return "neutral"

def mantener_formato(original, nuevo):
    """Ajusta el formato del nuevo nombre para que coincida con el original."""
    if original.istitle():  # Primera letra mayúscula
        return nuevo.title()
    elif original.isupper():  # Todo en mayúsculas
        return nuevo.upper()
    else:  # Todo en minúsculas
        return nuevo.lower()

def generar_nombre(original_name):
    """Genera un nombre ficticio manteniendo el género y la cantidad de palabras."""
    palabras = original_name.split()
    num_palabras = len(palabras)

    if original_name in nombre_map:
        return nombre_map[original_name]

    genero = detectar_genero(palabras[0])
    nuevo_nombre = []

    for _ in range(num_palabras):
        if genero == "male":
            nuevo_nombre.append(random.choice(list(nombres_masculinos)))
        else:
            nuevo_nombre.append(random.choice(list(nombres_femeninos)))

    nombre_final = " ".join(nuevo_nombre)
    nombre_final = mantener_formato(original_name, nombre_final)

    nombre_map[original_name] = nombre_final
    return nombre_final

def anonimizar_texto(texto):
    """Reemplaza nombres en el texto por versiones anonimizadas manteniendo consistencia."""
    palabras = texto.split()
    texto_anonimizado = []

    for i, palabra in enumerate(palabras):
        palabra_limpia = re.sub(r'[^A-Za-zÁÉÍÓÚáéíóúÑñ]', '', palabra).lower()
        if palabra_limpia in nombres_masculinos or palabra_limpia in nombres_femeninos:
            if palabra_limpia not in palabras_excluidas:
                nombre_anonimizado = generar_nombre(palabra)
                texto_anonimizado.append(nombre_anonimizado)
                palabra_anterior = palabras[i - 1] if i > 0 else ""
                palabra_posterior = palabras[i + 1] if i < len(palabras) - 1 else ""
                cambios.append((palabra, nombre_anonimizado, palabra_anterior, palabra_posterior))
            else:
                texto_anonimizado.append(palabra)
        else:
            texto_anonimizado.append(palabra)

    return " ".join(texto_anonimizado)

# Cargar datos desde el archivo Excel
df = pd.read_excel("MOSTRA_1.xlsx")

# Aplicar anonimización a la columna 'body'
df["body"] = df["body"].astype(str).apply(anonimizar_texto)

# Guardar el archivo anonimizado
df.to_excel("MOSTRA_1_anonimizado.xlsx", index=False)

# Crear DataFrame con los cambios y guardarlo en un archivo separado
cambios_df = pd.DataFrame(cambios, columns=["Original", "Anonimizado", "Palabra_Anterior", "Palabra_Posterior"])
cambios_df.to_excel("cambios_anonimizacion.xlsx", index=False)


In [8]:
import pandas as pd
import random
import re

# Función de preprocesamiento
def preprocesar_texto(texto):
    """Convierte el texto a minúsculas y elimina signos de puntuación."""
    texto = texto.lower()
    texto = re.sub(r'[^\w\sÁÉÍÓÚáéíóúÑñ]', '', texto)
    return texto

# Paso 1: Preprocesamiento
print("Iniciando preprocesamiento...")

# Cargar datos desde el archivo Excel
df = pd.read_excel("MOSTRA_1.xlsx")

# Aplicar preprocesamiento a la columna 'body'
df["body_preprocesado"] = df["body"].astype(str).apply(preprocesar_texto)

# Guardar el archivo preprocesado
df.to_excel("MOSTRA_1_preprocesado.xlsx", index=False)

print("Preprocesamiento completado. Archivo guardado como 'MOSTRA_1_preprocesado.xlsx'")

# Paso 2: Anonimización
print("Iniciando anonimización...")

# Cargar lista de nombres desde el archivo CSV
nombres_df = pd.read_csv("2024_pad_m_nom_sexe.csv")

# Filtrar nombres por género (convertir a minúsculas y eliminar espacios extra)
nombres_masculinos = set(nombres_df[nombres_df["SEXE"] == 2]["NOM"].str.strip().str.lower().tolist())
nombres_femeninos = set(nombres_df[nombres_df["SEXE"] == 1]["NOM"].str.strip().str.lower().tolist())

# Lista de palabras comunes que no deben anonimizarse
palabras_excluidas = {"bo","d'una","dan","justo","salud","li","leo","te","ella", "el", "los", "las", "nosotros", "vosotros",
                      "usted", "ustedes", "ellos", "ellas", "su", "sus", "un", "una", "unos", "unas", "nada", "mira", "duna"}

# Diccionario para mantener consistencia de nombres inventados
nombre_map = {}
cambios = []  # Lista para registrar los cambios

def detectar_genero(nombre):
    """Detecta si un nombre es masculino o femenino basado en la lista cargada."""
    nombre = nombre.lower()
    if nombre in nombres_masculinos:
        return "male"
    elif nombre in nombres_femeninos:
        return "female"
    return "neutral"

def mantener_formato(original, nuevo):
    """Ajusta el formato del nuevo nombre para que coincida con el original."""
    if original.istitle():  # Primera letra mayúscula
        return nuevo.title()
    elif original.isupper():  # Todo en mayúsculas
        return nuevo.upper()
    else:  # Todo en minúsculas
        return nuevo.lower()

def generar_nombre(original_name):
    """Genera un nombre ficticio manteniendo el género y la cantidad de palabras."""
    palabras = original_name.split()
    num_palabras = len(palabras)

    if original_name in nombre_map:
        return nombre_map[original_name]

    genero = detectar_genero(palabras[0])
    nuevo_nombre = []

    for _ in range(num_palabras):
        if genero == "male":
            nuevo_nombre.append(random.choice(list(nombres_masculinos)))
        else:
            nuevo_nombre.append(random.choice(list(nombres_femeninos)))

    nombre_final = " ".join(nuevo_nombre)
    nombre_final = mantener_formato(original_name, nombre_final)

    nombre_map[original_name] = nombre_final
    return nombre_final

def anonimizar_texto(texto):
    """Reemplaza nombres en el texto por versiones anonimizadas manteniendo consistencia."""
    # Convert the input to a string before calling split
    texto = str(texto)
    palabras = texto.split()
    texto_anonimizado = []

    for i, palabra in enumerate(palabras):
        palabra_limpia = re.sub(r'[^A-Za-zÁÉÍÓÚáéíóúÑñ]', '', palabra).lower()
        if palabra_limpia in nombres_masculinos or palabra_limpia in nombres_femeninos:
            if palabra_limpia not in palabras_excluidas:
                nombre_anonimizado = generar_nombre(palabra)
                texto_anonimizado.append(nombre_anonimizado)
                palabra_anterior = palabras[i - 1] if i > 0 else ""
                palabra_posterior = palabras[i + 1] if i < len(palabras) - 1 else ""
                cambios.append((palabra, nombre_anonimizado, palabra_anterior, palabra_posterior))
            else:
                texto_anonimizado.append(palabra)
        else:
            texto_anonimizado.append(palabra)

    return " ".join(texto_anonimizado)
# Cargar el archivo preprocesado
df_preprocesado = pd.read_excel("MOSTRA_1_preprocesado.xlsx")

# Aplicar anonimización a la columna 'body_preprocesado'
df_preprocesado["body_anonimizado"] = df_preprocesado["body_preprocesado"].apply(anonimizar_texto)

# Guardar el archivo anonimizado
df_preprocesado.to_excel("MOSTRA_1_anonimizado.xlsx", index=False)

# Crear DataFrame con los cambios y guardarlo en un archivo separado
cambios_df = pd.DataFrame(cambios, columns=["Original", "Anonimizado", "Palabra_Anterior", "Palabra_Posterior"])
cambios_df.to_excel("cambios_anonimizacion.xlsx", index=False)

print("Anonimización completada. Archivos guardados como 'MOSTRA_1_anonimizado.xlsx' y 'cambios_anonimizacion.xlsx'")


Iniciando preprocesamiento...
Preprocesamiento completado. Archivo guardado como 'MOSTRA_1_preprocesado.xlsx'
Iniciando anonimización...
Anonimización completada. Archivos guardados como 'MOSTRA_1_anonimizado.xlsx' y 'cambios_anonimizacion.xlsx'
